In [15]:
import pandas as pd
import numpy as np
import re

In [21]:
players_df = pd.read_csv("all_international_players_info.csv")

# Odi formatting

In [2]:
odi_batting_df = pd.read_csv("odi_batting_data.csv")
odi_bowling_df = pd.read_csv("odi_bowling_data.csv")

In [30]:
odi_batting_df.head()

,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,4s,6s,Strike Rate,Match id
0,Bangladesh,1,Tamim Iqbal,1,40,46,5,0,86.95,2937
1,Bangladesh,2,Imrul Kayes,1,23,37,2,0,62.16,2937
2,Bangladesh,3,Mohammad Ashraful,1,75,94,6,0,79.78,2937
3,Bangladesh,4,Raqibul Hasan,1,0,6,0,0,0.00,2937
4,Bangladesh,5,Shakib Al Hasan,1,1,7,0,0,14.28,2937


In [29]:
odi_batting_df = odi_batting_df.drop(columns=['M'])

In [5]:
odi_batting_df['Dismissal'] = odi_batting_df['Dismissal'].apply(lambda row:0 if row=='not out' else 1)

In [13]:
def clean_string(s):
    s = re.sub(r'[^\x00-\x7F]+', '', s)  # Remove non-ASCII characters
    s = re.sub(r'\(c\)|\(†\)', '', s)   # Remove (c) or (†)
    return s

In [133]:
odi_batting_df['Batsman Name'] = odi_batting_df['Batsman Name'].apply(lambda x:clean_string(x))

In [134]:
odi_batting_df = odi_batting_df.replace('-',0)

In [135]:
odi_batting_df['Batting Position'] = odi_batting_df['Batting Position'].astype(int)
odi_batting_df['Runs'] = odi_batting_df['Runs'].astype(int)
odi_batting_df['Balls'] = odi_batting_df['Balls'].astype(int)
odi_batting_df['4s'] = odi_batting_df['4s'].astype(int)
odi_batting_df['6s'] = odi_batting_df['6s'].astype(int)
odi_batting_df['Match id'] = odi_batting_df['Match id'].astype(int)
odi_batting_df['Strike Rate'] = odi_batting_df['Strike Rate'].astype(float)

In [136]:
odi_batting_df['Hundreds'] = odi_batting_df['Runs'].apply(lambda row:1 if row>=100 else 0)
odi_batting_df['Fifties'] = odi_batting_df['Runs'].apply(lambda row:1 if row>=50 and row<100 else 0)

In [137]:
odi_batting_df.head()

,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,4s,6s,Strike Rate,Match id,Hundreds,Fifties
0,Bangladesh,1,Tamim Iqbal,1,40,46,5,0,86.95,2937,0,0
1,Bangladesh,2,Imrul Kayes,1,23,37,2,0,62.16,2937,0,0
2,Bangladesh,3,Mohammad Ashraful,1,75,94,6,0,79.78,2937,0,1
3,Bangladesh,4,Raqibul Hasan,1,0,6,0,0,0.00,2937,0,0
4,Bangladesh,5,Shakib Al Hasan,1,1,7,0,0,14.28,2937,0,0


In [138]:
grouped_odi_batting_stats = odi_batting_df.groupby(['Batsman Name','teamInnings']).agg(
    Batting_pos=('Batting Position','sum'),
    Runs_Scored=('Runs', 'sum'),
    Fours_Scored=('4s', 'sum'),
    Sixes_scored=('6s', 'sum'),
    Balls_Faced=('Balls', 'sum'),
    Total_Matches=('Match id', 'nunique'),
    Total_dismissals = ('Dismissal','sum'),
    Total_hundreds = ('Hundreds','sum'),
    Total_fifties = ('Fifties','sum')
).reset_index()

In [139]:
grouped_odi_batting_stats = grouped_odi_batting_stats.sort_values(by = "Batsman Name")
grouped_odi_batting_stats.head()

,Batsman Name,teamInnings,Batting_pos,Runs_Scored,Fours_Scored,Sixes_scored,Balls_Faced,Total_Matches,Total_dismissals,Total_hundreds,Total_fifties
0,AB de Villiers,South Africa,538,6343,510,150,5770,127,100,20,33
1,Aamer Yamin,Pakistan,25,95,8,5,96,3,1,0,1
2,Aamir Kaleem,Oman,6,13,1,0,16,1,1,0,0
3,Aarif Sheikh,Nepal,37,94,7,0,176,6,5,0,0
4,Aaron Finch,Australia,184,4527,449,112,5038,112,110,15,24


In [140]:
grouped_odi_batting_stats.insert(2,"Batting Position",round(grouped_odi_batting_stats['Batting_pos']/grouped_odi_batting_stats['Total_Matches']).astype(int),True)

In [141]:
grouped_odi_batting_stats = grouped_odi_batting_stats.drop(columns=['Batting_pos'])

In [142]:
grouped_odi_batting_stats[grouped_odi_batting_stats['Batsman Name']=='Virat Kohli']

,Batsman Name,teamInnings,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Balls_Faced,Total_Matches,Total_dismissals,Total_hundreds,Total_fifties
1001,Virat Kohli,India,3,10964,1024,115,11624,216,182,42,50


In [143]:
grouped_odi_batting_stats = grouped_odi_batting_stats.rename(columns={"teamInnings":"Team Name","Batsman Name":"Player Name"})

In [144]:
grouped_odi_batting_stats['Strike_rate'] = ((grouped_odi_batting_stats['Runs_Scored']/grouped_odi_batting_stats['Balls_Faced'])*100).round(2)

In [145]:
grouped_odi_batting_stats['Batting Average'] = (grouped_odi_batting_stats['Runs_Scored']/grouped_odi_batting_stats['Total_dismissals']).round(2)

In [148]:
grouped_odi_batting_stats.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
0,AB de Villiers,South Africa,4,6343,20,33,109.93,63.43
1,Aamer Yamin,Pakistan,8,95,0,1,98.96,95.00
2,Aamir Kaleem,Oman,6,13,0,0,81.25,13.00
3,Aarif Sheikh,Nepal,6,94,0,0,53.41,18.80
4,Aaron Finch,Australia,2,4527,15,24,89.86,41.15


In [147]:
grouped_odi_batting_stats = grouped_odi_batting_stats.drop(columns=["Balls_Faced","Total_Matches","Total_dismissals","Fours_Scored","Sixes_scored"])

In [105]:
odi_bowling_df.head()

,match,teamInnings,Bowler Name,Over,Runs,Wickets,Economy,fours,sixes,Match Id
0,Bangladesh VsSri Lanka,Sri Lanka,Nuwan Kulasekara,10.0,46,1,4.60,2,0,2937
1,Bangladesh VsSri Lanka,Sri Lanka,Chanaka Welegedara,8.0,39,0,4.87,5,0,2937
2,Bangladesh VsSri Lanka,Sri Lanka,Tillakaratne Dilshan,3.0,16,1,5.33,1,0,2937
3,Bangladesh VsSri Lanka,Sri Lanka,Suranga Lakmal,9.0,63,2,7.00,8,2,2937
4,Bangladesh VsSri Lanka,Sri Lanka,Suraj Randiv,10.0,51,2,5.10,3,1,2937


In [107]:
odi_bowling_df = odi_bowling_df.replace("-",0)

In [108]:
odi_bowling_df['Runs'] = odi_bowling_df['Runs'].astype(int)
odi_bowling_df['Wickets'] = odi_bowling_df['Wickets'].astype(int)
odi_bowling_df['fours'] = odi_bowling_df['fours'].astype(int)
odi_bowling_df['sixes'] = odi_bowling_df['sixes'].astype(int)

In [110]:
grouped_odi_bowling_stats = odi_bowling_df.groupby(['Bowler Name','teamInnings']).agg(
    Runs_Conceded=('Runs','sum'),
    Overs_Bowled=('Over','sum'),
    Total_Wickets_Taken=('Wickets','sum'),
    Fours_Conceded=('fours','sum'),
    Sixes_Conceded=('sixes','sum'),
    Total_Matches=('Match Id', 'nunique'),
).reset_index()

In [112]:
def calculate_balls_bowled(overs):
    integer_part = int(overs)
    decimal_part = overs - integer_part
    total_overs = integer_part*6 + (decimal_part*10)
    return int(total_overs)

In [113]:
grouped_odi_bowling_stats['Total_Balls_bowled'] = grouped_odi_bowling_stats['Overs_Bowled'].apply(lambda over:calculate_balls_bowled(over))

In [117]:
grouped_odi_bowling_stats['Economy'] = grouped_odi_bowling_stats['Runs_Conceded']/grouped_odi_bowling_stats['Overs_Bowled']
grouped_odi_bowling_stats['Economy'] = grouped_odi_bowling_stats['Economy'].round(1)grouped_odi_bowling_stats['Bowling Average'] = (grouped_odi_bowling_stats['Runs_Conceded']/grouped_odi_bowling_stats['Total_Wickets_Taken']).round(2)

In [119]:
grouped_odi_bowling_stats['Bowling Average'] = (grouped_odi_bowling_stats['Runs_Conceded']/grouped_odi_bowling_stats['Total_Wickets_Taken']).round(2)

In [122]:
grouped_odi_bowling_stats['Bowling Strike Rate'] = (grouped_odi_bowling_stats['Total_Balls_bowled']/grouped_odi_bowling_stats['Total_Wickets_Taken']).round(2)

In [124]:
grouped_odi_bowling_stats[grouped_odi_bowling_stats['Bowler Name']=='Lasith Malinga']

,Bowler Name,teamInnings,Runs_Conceded,Overs_Bowled,Total_Wickets_Taken,Fours_Conceded,Sixes_Conceded,Total_Matches,Total_Balls_bowled,Economy,Bowling Average,Bowling Strike Rate
401,Lasith Malinga,Sri Lanka,6991,1268.8,241,705,60,153,7616,5.5,29.01,31.6


In [125]:
grouped_odi_bowling_stats = grouped_odi_bowling_stats.rename(columns={"teamInnings":"Team Name","Bowler Name":"Player Name"})

In [154]:
grouped_odi_bowling_stats.head()

,Player Name,Team Name,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate
0,AB de Villiers,South Africa,180,7,6.0,25.71,25.71
1,Aamer Yamin,Pakistan,154,2,5.9,77.00,78.00
2,Aamir Kaleem,Oman,54,2,4.3,27.00,38.50
3,Aarif Sheikh,Nepal,10,0,5.0,0.00,0.00
4,Aaron Finch,Australia,239,4,5.5,59.75,66.50


In [153]:
grouped_odi_bowling_stats = grouped_odi_bowling_stats.replace(np.inf,0.0)

In [130]:
grouped_odi_bowling_stats = grouped_odi_bowling_stats.drop(columns=['Overs_Bowled','Fours_Conceded','Sixes_Conceded','Total_Matches','Total_Balls_bowled'])

In [156]:
semi_merged_odi_df = pd.merge(grouped_odi_batting_stats,grouped_odi_bowling_stats,on='Player Name',how='outer')
semi_merged_odi_df.head()

,Player Name,Team Name_x,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Team Name_y,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate
0,AB de Villiers,South Africa,4.0,6343.0,20.0,33.0,109.93,63.43,South Africa,180.0,7.0,6.0,25.71,25.71
1,Aamer Yamin,Pakistan,8.0,95.0,0.0,1.0,98.96,95.00,Pakistan,154.0,2.0,5.9,77.00,78.00
2,Aamir Kaleem,Oman,6.0,13.0,0.0,0.0,81.25,13.00,Oman,54.0,2.0,4.3,27.00,38.50
3,Aarif Sheikh,Nepal,6.0,94.0,0.0,0.0,53.41,18.80,Nepal,10.0,0.0,5.0,0.00,0.00
4,Aaron Finch,Australia,2.0,4527.0,15.0,24.0,89.86,41.15,Australia,239.0,4.0,5.5,59.75,66.50


In [157]:
semi_merged_odi_df = semi_merged_odi_df.drop(columns=['Team Name_y'])
semi_merged_odi_df = semi_merged_odi_df.rename(columns = {"Team Name_x":"Team Name"})

In [161]:
semi_merged_odi_df = semi_merged_odi_df.replace(np.nan,0.0)

In [175]:
semi_merged_odi_df.shape

(1094, 13)

In [201]:
players_df = players_df.rename(columns={"Name":"Player Name","Team":"Team Name",'Role':'Playing Role'})
players_df.head()

,Player Name,Team Name,Playing Role,Batting Style,Bowling Style
0,Aakash Chopra,India,Batter,Right hand Bat,"Right arm Medium, Right arm Offbreak"
1,Aaliyah Alleyne,West Indies,Bowler,Right hand Bat,Right arm Medium
2,Aaliyah Williams,West Indies,Allrounder,Right hand Bat,Right arm Medium
3,Aamer Jamal,Pakistan,Allrounder,Right hand Bat,Right arm Medium
4,Aamer Malik,Pakistan,Wicketkeeper Batter,Right hand Bat,Right arm Fast medium


In [202]:
merged_odi_df = pd.merge(semi_merged_odi_df,players_df,on = ['Player Name','Team Name'],how='left')

In [203]:
merged_odi_df[merged_odi_df['Player Name']=='Virat Kohli']

,Player Name,Team Name,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate,Playing Role,Batting Style,Bowling Style
1011,Virat Kohli,India,3.0,10964.0,42.0,50.0,94.32,60.24,622.0,4.0,6.2,155.5,150.25,Top order Batter,Right hand Bat,Right arm Medium


In [204]:
merged_odi_df['Batting Position'] = merged_odi_df['Batting Position'].astype(int)
merged_odi_df['Runs_Scored'] = merged_odi_df['Runs_Scored'].astype(int)
merged_odi_df['Total_hundreds'] = merged_odi_df['Total_hundreds'].astype(int)
merged_odi_df['Total_fifties'] = merged_odi_df['Total_fifties'].astype(int)
merged_odi_df['Runs_Conceded'] = merged_odi_df['Runs_Conceded'].astype(int)
# merged_odi_df['Runs_Conceded'] = merged_odi_df['Runs_Conceded'].astype(int)
merged_odi_df['Total_Wickets_Taken'] = merged_odi_df['Total_Wickets_Taken'].astype(int)

In [205]:
merged_odi_df.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Average,Bowling Strike Rate,Playing Role,Batting Style,Bowling Style
0,AB de Villiers,South Africa,4,6343,20,33,109.93,63.43,180,7,6.0,25.71,25.71,Wicketkeeper Batter,Right hand Bat,Right arm Medium
1,Aamer Yamin,Pakistan,8,95,0,1,98.96,95.00,154,2,5.9,77.00,78.00,Allrounder,Right hand Bat,Right arm Medium
2,Aamir Kaleem,Oman,6,13,0,0,81.25,13.00,54,2,4.3,27.00,38.50,NaN,NaN,NaN
3,Aarif Sheikh,Nepal,6,94,0,0,53.41,18.80,10,0,5.0,0.00,0.00,NaN,NaN,NaN
4,Aaron Finch,Australia,2,4527,15,24,89.86,41.15,239,4,5.5,59.75,66.50,Top order Batter,Right hand Bat,Slow Left arm Orthodox


In [206]:
new_odi_player_order = ['Player Name','Team Name','Playing Role','Batting Style','Batting Position','Runs_Scored','Batting Average','Strike_rate','Total_hundreds','Total_fifties','Bowling Style','Runs_Conceded','Total_Wickets_Taken','Economy','Bowling Strike Rate','Bowling Average']

In [207]:
merged_odi_df = merged_odi_df.reindex(columns=new_odi_player_order)

In [208]:
merged_odi_df['Team Name'].unique()

array(['South Africa', 'Pakistan', 'Oman', 'Nepal', 'Australia',
       'United States of America', 'New Zealand', 'United Arab Emirates',
       'Afghanistan', 'Bangladesh', 'India', 'Netherlands', 'England',
       'West Indies', 'Scotland', 'Hong Kong', 'Sri Lanka', 'Ireland',
       'Papua New Guinea', 'Kenya', 'Zimbabwe', 'Canada', 'Namibia', 0.0],
      dtype=object)

In [209]:
team_list = ['India','South Africa','Pakistan','Australia','New Zealand','Afghanistan','Bangladesh','England','West Indies','Sri Lanka','Ireland','Zimbabwe']
len(team_list)

12

In [233]:
merged_odi_df.shape

(726, 16)

In [217]:
# merged_odi_df['Player Name'] = merged_odi_df['Player Name'].drop_duplicates()

In [232]:
merged_odi_df = merged_odi_df[merged_odi_df['Team Name'].isin(team_list)]
len(merged_odi_df['Player Name'].unique())

726

In [221]:
merged_odi_df = merged_odi_df.drop_duplicates(subset=["Player Name","Team Name"])

In [231]:
merged_odi_df = merged_odi_df[merged_odi_df['Player Name'].notna()]

In [234]:
merged_odi_df.to_csv('merged_odi_with_stats_data.csv',index=False)

# T20 data processing

In [235]:
t20_batting_df = pd.read_csv("t20_batting_data.csv")
t20_bowling_df = pd.read_csv("t20_bowling_data.csv")

In [238]:
t20_batting_df = t20_batting_df.drop(columns=['M'])

In [239]:
t20_batting_df['Dismissal'] = t20_batting_df['Dismissal'].apply(lambda row:0 if row=='not out' else 1)

In [240]:
t20_batting_df['Batsman Name'] = t20_batting_df['Batsman Name'].apply(lambda x:clean_string(x))

In [242]:
t20_batting_df = t20_batting_df.replace('-',0)

In [243]:
t20_batting_df['Batting Position'] = t20_batting_df['Batting Position'].astype(int)
t20_batting_df['Runs'] = t20_batting_df['Runs'].astype(int)
t20_batting_df['Balls'] = t20_batting_df['Balls'].astype(int)
t20_batting_df['4s'] = t20_batting_df['4s'].astype(int)
t20_batting_df['6s'] = t20_batting_df['6s'].astype(int)
t20_batting_df['Match id'] = t20_batting_df['Match id'].astype(int)
t20_batting_df['Strike Rate'] = t20_batting_df['Strike Rate'].astype(float)

In [245]:
t20_batting_df['Hundreds'] = t20_batting_df['Runs'].apply(lambda row:1 if row>=100 else 0)
t20_batting_df['Fifties'] = t20_batting_df['Runs'].apply(lambda row:1 if row>=50 and row<100 else 0)

In [256]:
grouped_t20_batting_stats = t20_batting_df.groupby(['Batsman Name','teamInnings']).agg(
    Batting_pos=('Batting Position','sum'),
    Runs_Scored=('Runs', 'sum'),
    Fours_Scored=('4s', 'sum'),
    Sixes_scored=('6s', 'sum'),
    Balls_Faced=('Balls', 'sum'),
    Total_Matches=('Match id', 'nunique'),
    Total_dismissals = ('Dismissal','sum'),
    Total_hundreds = ('Hundreds','sum'),
    Total_fifties = ('Fifties','sum')
).reset_index()

In [257]:
grouped_t20_batting_stats.head()

,Batsman Name,teamInnings,Batting_pos,Runs_Scored,Fours_Scored,Sixes_scored,Balls_Faced,Total_Matches,Total_dismissals,Total_hundreds,Total_fifties
0,AB de Villiers,South Africa,182,1275,103,49,926,52,46,0,7
1,Aahan Gopinath Achar,Singapore,22,2,0,0,2,2,0,0,0
2,Aamer Ikram,Portugal,18,58,4,4,73,4,4,0,0
3,Aamer Yamin,Pakistan,8,15,1,1,6,1,0,0,0
4,Aamir Kaleem,Oman,106,325,26,9,306,22,18,0,0


In [258]:
grouped_t20_batting_stats.insert(2,"Batting Position",round(grouped_t20_batting_stats['Batting_pos']/grouped_t20_batting_stats['Total_Matches']).astype(int),True)

In [259]:
grouped_t20_batting_stats = grouped_t20_batting_stats.drop(columns=['Batting_pos'])

In [268]:
grouped_t20_batting_stats[grouped_t20_batting_stats['Player Name']=="Virat Kohli"]

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Balls_Faced,Total_Matches,Total_dismissals,Total_hundreds,Total_fifties,Strike_rate,Batting Average
1649,Virat Kohli,India,3,2633,247,71,1907,70,50,0,24,138.07,52.66


In [261]:
grouped_t20_batting_stats = grouped_t20_batting_stats.rename(columns={"teamInnings":"Team Name","Batsman Name":"Player Name"})

In [262]:
grouped_t20_batting_stats['Strike_rate'] = ((grouped_t20_batting_stats['Runs_Scored']/grouped_t20_batting_stats['Balls_Faced'])*100).round(2)

In [266]:
grouped_t20_batting_stats['Batting Average'] = (grouped_t20_batting_stats['Runs_Scored']/grouped_t20_batting_stats['Total_dismissals']).round(2)

In [269]:
grouped_t20_batting_stats = grouped_t20_batting_stats.drop(columns=["Balls_Faced","Total_Matches","Total_dismissals"])

In [270]:
grouped_t20_batting_stats.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
0,AB de Villiers,South Africa,4,1275,103,49,0,7,137.69,27.72
1,Aahan Gopinath Achar,Singapore,11,2,0,0,0,0,100.00,inf
2,Aamer Ikram,Portugal,4,58,4,4,0,0,79.45,14.50
3,Aamer Yamin,Pakistan,8,15,1,1,0,0,250.00,inf
4,Aamir Kaleem,Oman,5,325,26,9,0,0,106.21,18.06


In [271]:
t20_bowling_df = t20_bowling_df.replace("-",0)

In [272]:
t20_bowling_df['Runs'] = t20_bowling_df['Runs'].astype(int)
t20_bowling_df['Wickets'] = t20_bowling_df['Wickets'].astype(int)
t20_bowling_df['fours'] = t20_bowling_df['fours'].astype(int)
t20_bowling_df['sixes'] = t20_bowling_df['sixes'].astype(int)

In [426]:
grouped_t20_bowling_stats = t20_bowling_df.groupby(['Bowler Name','teamInnings']).agg(
    Runs_Conceded=('Runs','sum'),
    Overs_Bowled=('Over','sum'),
    Total_Wickets_Taken=('Wickets','sum'),
    Total_maiden_overs = ('Maiden','sum'),
    Fours_Conceded=('fours','sum'),
    Sixes_Conceded=('sixes','sum'),
    Total_Matches=('Match Id', 'nunique'),
).reset_index()

In [427]:
grouped_t20_bowling_stats['Total_Balls_bowled'] = grouped_t20_bowling_stats['Overs_Bowled'].apply(lambda over:calculate_balls_bowled(over))

In [429]:
grouped_t20_bowling_stats['Economy'] = grouped_t20_bowling_stats['Runs_Conceded']/grouped_t20_bowling_stats['Overs_Bowled']
grouped_t20_bowling_stats['Economy'] = grouped_t20_bowling_stats['Economy'].round(1)

In [285]:
grouped_t20_bowling_stats['Bowling Average'] = (grouped_t20_bowling_stats['Runs_Conceded']/grouped_t20_bowling_stats['Total_Wickets_Taken']).round(2)

In [430]:
grouped_t20_bowling_stats['Bowling Strike Rate'] = (grouped_t20_bowling_stats['Total_Balls_bowled']/grouped_t20_bowling_stats['Total_Wickets_Taken']).round(2)

In [431]:
grouped_t20_bowling_stats = grouped_t20_bowling_stats.rename(columns={"teamInnings":"Team Name","Bowler Name":"Player Name"})

In [432]:
grouped_t20_bowling_stats[grouped_t20_bowling_stats['Player Name']=='Lasith Malinga']

,Player Name,Team Name,Runs_Conceded,Overs_Bowled,Total_Wickets_Taken,Total_maiden_overs,Fours_Conceded,Sixes_Conceded,Total_Matches,Total_Balls_bowled,Economy,Bowling Strike Rate
604,Lasith Malinga,Sri Lanka,1561,216.9,82,1,135,24,59,1305,7.2,15.91


In [295]:
grouped_t20_bowling_stats = grouped_t20_bowling_stats.replace(np.inf,0.0)

In [297]:
grouped_t20_bowling_stats = grouped_t20_bowling_stats.drop(columns=['Overs_Bowled','Total_Matches','Total_Balls_bowled'])

In [300]:
grouped_t20_batting_stats = grouped_t20_batting_stats.replace(np.inf,0.0)

In [305]:
grouped_t20_batting_stats.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
0,AB de Villiers,South Africa,4,1275,103,49,0,7,137.69,27.72
1,Aahan Gopinath Achar,Singapore,11,2,0,0,0,0,100.00,0.00
2,Aamer Ikram,Portugal,4,58,4,4,0,0,79.45,14.50
3,Aamer Yamin,Pakistan,8,15,1,1,0,0,250.00,0.00
4,Aamir Kaleem,Oman,5,325,26,9,0,0,106.21,18.06


In [298]:
grouped_t20_bowling_stats.head()

,Player Name,Team Name,Runs_Conceded,Total_Wickets_Taken,Fours_Conceded,Sixes_Conceded,Economy,Bowling Average,Bowling Strike Rate
0,Aahan Gopinath Achar,Singapore,44,2,2,1,7.3,22.00,18.00
1,Aamer Yamin,Pakistan,37,2,3,1,6.2,18.50,18.00
2,Aamir Kaleem,Oman,388,22,29,11,7.2,17.64,14.73
3,Aaron Finch,Australia,27,0,3,2,13.5,0.00,0.00
4,Aaron Jones,United States of America,28,2,0,2,4.0,14.00,21.00


In [311]:
semi_merged_t20_df = pd.merge(grouped_t20_batting_stats,grouped_t20_bowling_stats,on=['Player Name','Team Name'],how='outer')
# semi_merged_t20_df.head()

In [313]:
semi_merged_t20_df = semi_merged_t20_df.fillna(0)

In [318]:
semi_merged_t20_df[semi_merged_t20_df['Player Name']=='Kane Williamson']

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Fours_Conceded,Sixes_Conceded,Economy,Bowling Average,Bowling Strike Rate
771,Kane Williamson,New Zealand,3.0,1488.0,157.0,26.0,0.0,9.0,121.87,31.0,164.0,6.0,7.0,9.0,8.5,27.33,19.5


In [316]:
semi_merged_t20_df.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Fours_Conceded,Sixes_Conceded,Economy,Bowling Average,Bowling Strike Rate
0,AB de Villiers,South Africa,4.0,1275.0,103.0,49.0,0.0,7.0,137.69,27.72,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,Aahan Gopinath Achar,Singapore,11.0,2.0,0.0,0.0,0.0,0.0,100.00,0.00,44.0,2.0,2.0,1.0,7.3,22.00,18.00
2,Aamer Ikram,Portugal,4.0,58.0,4.0,4.0,0.0,0.0,79.45,14.50,0.0,0.0,0.0,0.0,0.0,0.00,0.00
3,Aamer Yamin,Pakistan,8.0,15.0,1.0,1.0,0.0,0.0,250.00,0.00,37.0,2.0,3.0,1.0,6.2,18.50,18.00
4,Aamir Kaleem,Oman,5.0,325.0,26.0,9.0,0.0,0.0,106.21,18.06,388.0,22.0,29.0,11.0,7.2,17.64,14.73


In [319]:
merged_t20_df = pd.merge(semi_merged_t20_df,players_df,on = ['Player Name','Team Name'],how='left')

In [322]:
merged_t20_df.head()

,Player Name,Team Name,Batting Position,Runs_Scored,Fours_Scored,Sixes_scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_Taken,Fours_Conceded,Sixes_Conceded,Economy,Bowling Average,Bowling Strike Rate,Playing Role,Batting Style,Bowling Style
0,AB de Villiers,South Africa,4,1275,103,49,0,7,137.69,27.72,0,0,0,0,0.0,0.00,0.00,Wicketkeeper Batter,Right hand Bat,Right arm Medium
1,Aahan Gopinath Achar,Singapore,11,2,0,0,0,0,100.00,0.00,44,2,2,1,7.3,22.00,18.00,NaN,NaN,NaN
2,Aamer Ikram,Portugal,4,58,4,4,0,0,79.45,14.50,0,0,0,0,0.0,0.00,0.00,NaN,NaN,NaN
3,Aamer Yamin,Pakistan,8,15,1,1,0,0,250.00,0.00,37,2,3,1,6.2,18.50,18.00,Allrounder,Right hand Bat,Right arm Medium
4,Aamir Kaleem,Oman,5,325,26,9,0,0,106.21,18.06,388,22,29,11,7.2,17.64,14.73,NaN,NaN,NaN


In [323]:
merged_t20_df['Batting Position'] = merged_t20_df['Batting Position'].astype(int)
merged_t20_df['Runs_Scored'] = merged_t20_df['Runs_Scored'].astype(int)
merged_t20_df['Total_hundreds'] = merged_t20_df['Total_hundreds'].astype(int)
merged_t20_df['Total_fifties'] = merged_t20_df['Total_fifties'].astype(int)
merged_t20_df['Runs_Conceded'] = merged_t20_df['Runs_Conceded'].astype(int)
merged_t20_df['Total_Wickets_Taken'] = merged_t20_df['Total_Wickets_Taken'].astype(int)
merged_t20_df['Fours_Scored'] = merged_t20_df['Fours_Scored'].astype(int)
merged_t20_df['Fours_Conceded'] = merged_t20_df['Fours_Conceded'].astype(int)
merged_t20_df['Sixes_scored'] = merged_t20_df['Sixes_scored'].astype(int)
merged_t20_df['Sixes_Conceded'] = merged_t20_df['Sixes_Conceded'].astype(int)

In [332]:
merged_t20_df = merged_t20_df[merged_t20_df['Team Name'].isin(team_list)]
merged_t20_df.head()

,Player Name,Team Name,Playing Role,Batting Style,Batting Position,Runs_Scored,Batting Average,Strike_rate,Total_hundreds,Total_fifties,Fours_Scored,Sixes_scored,Bowling Style,Runs_Conceded,Total_Wickets_Taken,Economy,Bowling Strike Rate,Bowling Average,Fours_Conceded,Sixes_Conceded
0,AB de Villiers,South Africa,Wicketkeeper Batter,Right hand Bat,4,1275,27.72,137.69,0,7,103,49,Right arm Medium,0,0,0.0,0.0,0.00,0,0
3,Aamer Yamin,Pakistan,Allrounder,Right hand Bat,8,15,0.00,250.00,0,0,1,1,Right arm Medium,37,2,6.2,18.0,18.50,3,1
6,Aaron Finch,Australia,Top order Batter,Right hand Bat,2,1841,38.35,155.62,2,11,177,88,Slow Left arm Orthodox,27,0,13.5,0.0,0.00,3,2
9,Aaron Phangiso,South Africa,Bowler,Right hand Bat,9,19,6.33,65.52,0,0,1,0,Slow Left arm Orthodox,443,20,7.8,17.1,22.15,27,24
10,Aaron Redmond,New Zealand,Top order Batter,Right hand Bat,2,25,8.33,92.59,0,0,1,1,Legbreak,0,0,0.0,0.0,0.00,0,0


In [330]:
new_t20_player_order = ['Player Name','Team Name','Playing Role','Batting Style','Batting Position','Runs_Scored','Batting Average','Strike_rate','Total_hundreds','Total_fifties','Fours_Scored','Sixes_scored','Bowling Style','Runs_Conceded','Total_Wickets_Taken','Economy','Bowling Strike Rate','Bowling Average','Fours_Conceded','Sixes_Conceded']

In [331]:
merged_t20_df = merged_t20_df.reindex(columns=new_t20_player_order)

In [333]:
merged_t20_df = merged_t20_df.drop_duplicates(subset=["Player Name","Team Name"])

In [337]:
merged_t20_df.shape

(744, 20)

In [336]:
merged_t20_df = merged_t20_df[merged_t20_df['Player Name'].notna()]

In [338]:
merged_t20_df.to_csv("merged_t20_stats_data.csv",index=False)

# Test Players Data Processing

In [359]:
test_batting_df = pd.read_csv("test_batting_data.csv")
test_bowling_df = pd.read_csv("test_bowling_data.csv")

In [368]:
test_batting_df.head()

,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,4s,6s,Strike Rate,Match id,match_name
0,Australia,1,Shane Watson,1,6,24,1,0,25.00,1945,Australia vs Pakistan
1,Australia,2,Phillip Hughes,1,0,10,0,0,0.00,1945,Australia vs Pakistan
2,Australia,3,Ricky Ponting,1,0,1,0,0,0.00,1945,Australia vs Pakistan
3,Australia,4,Michael Hussey,1,28,52,4,0,53.84,1945,Australia vs Pakistan
4,Australia,5,Michael Clarke,1,3,30,0,0,10.00,1945,Australia vs Pakistan


In [364]:
test_batting_df = test_batting_df.drop(columns=['M'])

In [365]:
test_batting_df['Dismissal'] = test_batting_df['Dismissal'].apply(lambda row:0 if row=='not out' else 1)

In [367]:
test_batting_df['Batsman Name'] = test_batting_df['Batsman Name'].apply(lambda x:clean_string(x))

In [369]:
test_batting_df = test_batting_df.replace('-',0)

In [370]:
test_batting_df['Batting Position'] = test_batting_df['Batting Position'].astype(int)
test_batting_df['Runs'] = test_batting_df['Runs'].astype(int)
test_batting_df['Balls'] = test_batting_df['Balls'].astype(int)
test_batting_df['4s'] = test_batting_df['4s'].astype(int)
test_batting_df['6s'] = test_batting_df['6s'].astype(int)
test_batting_df['Match id'] = test_batting_df['Match id'].astype(int)
test_batting_df['Strike Rate'] = test_batting_df['Strike Rate'].astype(float)

In [371]:
test_batting_df['Hundreds'] = test_batting_df['Runs'].apply(lambda row:1 if row>=100 else 0)
test_batting_df['Fifties'] = test_batting_df['Runs'].apply(lambda row:1 if row>=50 and row<100 else 0)

In [408]:
test_batting_df.head()

,teamInnings,Batting Position,Batsman Name,Dismissal,Runs,Balls,4s,6s,Strike Rate,Match id,match_name,Hundreds,Fifties
0,Australia,1,Shane Watson,1,6,24,1,0,25.00,1945,Australia vs Pakistan,0,0
1,Australia,2,Phillip Hughes,1,0,10,0,0,0.00,1945,Australia vs Pakistan,0,0
2,Australia,3,Ricky Ponting,1,0,1,0,0,0.00,1945,Australia vs Pakistan,0,0
3,Australia,4,Michael Hussey,1,28,52,4,0,53.84,1945,Australia vs Pakistan,0,0
4,Australia,5,Michael Clarke,1,3,30,0,0,10.00,1945,Australia vs Pakistan,0,0


In [410]:
grouped_test_batting_stats = test_batting_df.groupby(['Batsman Name']).agg(
    Team=('teamInnings', 'first'), # add this line to get the team for each player
    Batting_pos=('Batting Position','sum'),
    Runs_Scored=('Runs', 'sum'),
    Fours_Scored=('4s', 'sum'),
    Sixes_scored=('6s', 'sum'),
    Balls_Faced=('Balls', 'sum'),
    Total_Matches=('Match id', 'nunique'),
    Total_dismissals=('Dismissal','sum'),
    Total_hundreds=('Hundreds','sum'),
    Total_fifties=('Fifties','sum'),
    
    
).reset_index()


In [422]:
grouped_test_batting_stats.head()

,Player Name,Team,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
0,AB de Villiers,South Africa,5,5059,13,27,55.65,57.49
1,Aaron Finch,Australia,2,278,0,2,44.98,27.80
2,Aaron Redmond,New Zealand,3,26,0,0,54.17,13.00
3,Abdur Razzak,Bangladesh,9,119,0,0,72.56,11.90
4,Abdur Rehman,Pakistan,8,361,0,2,43.49,13.88


In [421]:
grouped_test_batting_stats[grouped_test_batting_stats['Player Name']=="Steven Smith"]

,Player Name,Team,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
505,Steven Smith,Australia,4,7164,26,28,55.6,62.84


In [419]:
players_df[players_df['Player Name']=='Steven Smith']

,Player Name,Team Name,Playing Role,Batting Style,Bowling Style
2591,Steven Smith,Australia,Middle order Batter,Right hand Bat,Legbreak Googly


In [413]:
grouped_test_batting_stats.insert(2,"Batting Position",round(grouped_test_batting_stats['Batting_pos']/grouped_test_batting_stats['Total_Innings_Batted']).astype(int),True)

In [414]:
grouped_test_batting_stats = grouped_test_batting_stats.drop(columns=['Batting_pos'])

In [415]:
grouped_test_batting_stats = grouped_test_batting_stats.rename(columns={"teamInnings":"Team Name","Batsman Name":"Player Name"})

In [416]:
grouped_test_batting_stats['Strike_rate'] = ((grouped_test_batting_stats['Runs_Scored']/grouped_test_batting_stats['Balls_Faced'])*100).round(2)

In [417]:
grouped_test_batting_stats['Batting Average'] = (grouped_test_batting_stats['Runs_Scored']/grouped_test_batting_stats['Total_dismissals']).round(2)

In [420]:
grouped_test_batting_stats = grouped_test_batting_stats.drop(columns=["Balls_Faced","Total_Matches","Total_dismissals","Fours_Scored","Sixes_scored","Total_Innings_Batted"])

In [423]:
test_bowling_dft = test_bowling_df.replace("-",0)

In [445]:
test_bowling_df.head(50)

,teamInnings,Bowler Name,Over,Maiden,Runs,Wickets,Economy,dots,fours,sixes,wides,no balls,Match id,match_name,team1,team2
0,Pakistan,Mohammad Asif,20.0,6,41,6,2.05,101,5,0,0,0,1945,Australia vs Pakistan,Australia,Pakistan
1,Pakistan,Mohammad Sami,12.0,4,27,3,2.25,59,4,0,0,1,1945,Australia vs Pakistan,Australia,Pakistan
2,Pakistan,Umar Gul,10.2,0,38,1,3.67,47,6,0,1,0,1945,Australia vs Pakistan,Australia,Pakistan
3,Pakistan,Danish Kaneria,2.0,0,18,0,9.00,5,2,1,0,0,1945,Australia vs Pakistan,Australia,Pakistan
4,Australia,Doug Bollinger,21.5,5,72,4,3.29,105,8,0,1,6,1945,Australia vs Pakistan,Australia,Pakistan
5,Australia,Peter Siddle,22.0,4,62,1,2.81,103,7,0,0,0,1945,Australia vs Pakistan,Australia,Pakistan
6,Australia,Mitchell Johnson,20.0,2,64,2,3.20,88,8,0,0,0,1945,Australia vs Pakistan,Australia,Pakistan
7,Australia,Shane Watson,17.0,4,40,2,2.35,85,6,0,0,0,1945,Australia vs Pakistan,Australia,Pakistan
8,Australia,Nathan Hauritz,16.0,3,77,1,4.81,69,11,1,0,4,1945,Australia vs Pakistan,Australia,Pakistan
9,Pakistan,Mohammad Asif,27.0,8,53,2,1.96,137,7,0,1,0,1945,Australia vs Pakistan,Australia,Pakistan


In [433]:
test_bowling_df['Runs'] = test_bowling_df['Runs'].astype(int)
test_bowling_df['Wickets'] = test_bowling_df['Wickets'].astype(int)
test_bowling_df['fours'] = test_bowling_df['fours'].astype(int)
test_bowling_df['sixes'] = test_bowling_df['sixes'].astype(int)

In [453]:
grouped_test_bowling_stats = test_bowling_df.groupby(['Bowler Name']).agg(
    Team=('teamInnings', 'first'), # add this line to get the team for each player
    Runs_Conceded=('Runs', 'sum'),
    Total_Wickets_taken=('Wickets','sum'),
    Fours_Conceded=('fours', 'sum'),
    Sixes_Conceded=('sixes', 'sum'),
    Overs_bowled=('Over','sum'),
    Total_Matches=('Match id', 'nunique'),
    Total_Innings_bowled=('teamInnings','count'),
    
).reset_index()

In [458]:
players_df[players_df['Player Name']=='Stuart Broad']

,Player Name,Team Name,Playing Role,Batting Style,Bowling Style
2594,Stuart Broad,England,Bowler,Left hand Bat,Right arm Fast medium


In [482]:
grouped_test_bowling_stats.head()

,Bowler Name,Team,Runs_Conceded,Total_Wickets_taken,Fours_Conceded,Sixes_Conceded,Overs_bowled,Total_Matches,Total_Innings_bowled,Total_Balls_bowled,Economy,Bowling Average,Bowling Strike Rate
0,AB de Villiers,South Africa,5,0,0,0,1.0,1,1,6,5.0,0.00,0.00
1,Aaron Finch,Australia,8,0,0,0,2.0,1,1,12,4.0,0.00,0.00
2,Aaron Redmond,New Zealand,18,0,3,0,5.0,1,1,30,3.6,0.00,0.00
3,Abdur Razzak,Bangladesh,1112,21,105,18,314.3,8,13,1887,3.5,52.95,89.86
4,Abdur Rehman,Pakistan,2558,88,224,46,1022.2,20,39,6134,2.5,29.07,69.70


In [485]:
grouped_test_bowling_stats[grouped_test_bowling_stats['Bowler Name']=='James Anderson']

,Bowler Name,Team,Runs_Conceded,Total_Wickets_taken,Economy,Bowling Average,Bowling Strike Rate
146,James Anderson,England,10448,429,2.7,24.35,54.83


In [464]:
grouped_test_bowling_stats['Total_Balls_bowled'] = grouped_test_bowling_stats['Overs_bowled'].apply(lambda over:calculate_balls_bowled(over))

In [467]:
grouped_test_bowling_stats['Economy'] = grouped_test_bowling_stats['Runs_Conceded']/grouped_test_bowling_stats['Overs_bowled']
grouped_test_bowling_stats['Economy'] = grouped_test_bowling_stats['Economy'].round(1)

In [472]:
grouped_test_bowling_stats['Bowling Average'] = (grouped_test_bowling_stats['Runs_Conceded']/grouped_test_bowling_stats['Total_Wickets_taken']).round(2)

In [478]:
grouped_test_bowling_stats['Bowling Strike Rate'] = (grouped_test_bowling_stats['Total_Balls_bowled']/grouped_test_bowling_stats['Total_Wickets_taken']).round(2)

In [481]:
grouped_test_bowling_stats = grouped_test_bowling_stats.replace(np.inf,0.0)

In [484]:
grouped_test_bowling_stats = grouped_test_bowling_stats.drop(columns=['Overs_bowled','Total_Matches','Total_Balls_bowled',"Fours_Conceded","Sixes_Conceded","Total_Matches","Total_Innings_bowled","Total_Balls_bowled"])

In [488]:
grouped_test_bowling_stats = grouped_test_bowling_stats.rename(columns={"Bowler Name":"Player Name"})

In [486]:
grouped_test_batting_stats.head()

,Player Name,Team,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average
0,AB de Villiers,South Africa,5,5059,13,27,55.65,57.49
1,Aaron Finch,Australia,2,278,0,2,44.98,27.80
2,Aaron Redmond,New Zealand,3,26,0,0,54.17,13.00
3,Abdur Razzak,Bangladesh,9,119,0,0,72.56,11.90
4,Abdur Rehman,Pakistan,8,361,0,2,43.49,13.88


In [489]:
grouped_test_bowling_stats.head()

,Player Name,Team,Runs_Conceded,Total_Wickets_taken,Economy,Bowling Average,Bowling Strike Rate
0,AB de Villiers,South Africa,5,0,5.0,0.00,0.00
1,Aaron Finch,Australia,8,0,4.0,0.00,0.00
2,Aaron Redmond,New Zealand,18,0,3.6,0.00,0.00
3,Abdur Razzak,Bangladesh,1112,21,3.5,52.95,89.86
4,Abdur Rehman,Pakistan,2558,88,2.5,29.07,69.70


In [490]:
semi_merged_test_df = pd.merge(grouped_test_batting_stats,grouped_test_bowling_stats,on=['Player Name','Team'],how='outer')

In [496]:
semi_merged_test_df = semi_merged_test_df.rename(columns={"Team":"Team Name"})

In [494]:
semi_merged_test_df = semi_merged_test_df.fillna(0)

In [497]:
merged_test_df = pd.merge(semi_merged_test_df,players_df,on = ['Player Name','Team Name'],how='left')

In [499]:
merged_test_df.shape

(604, 16)

In [505]:
merged_test_df.head(2)

,Player Name,Team Name,Batting Position,Runs_Scored,Total_hundreds,Total_fifties,Strike_rate,Batting Average,Runs_Conceded,Total_Wickets_taken,Economy,Bowling Average,Bowling Strike Rate,Playing Role,Batting Style,Bowling Style
0,AB de Villiers,South Africa,5,5059,13,27,55.65,57.49,5,0,5.0,0.0,0.0,Wicketkeeper Batter,Right hand Bat,Right arm Medium
1,Aaron Finch,Australia,2,278,0,2,44.98,27.80,8,0,4.0,0.0,0.0,Top order Batter,Right hand Bat,Slow Left arm Orthodox


In [504]:
merged_test_df['Batting Position'] = merged_test_df['Batting Position'].astype(int)
merged_test_df['Runs_Scored'] = merged_test_df['Runs_Scored'].astype(int)
merged_test_df['Total_hundreds'] = merged_test_df['Total_hundreds'].astype(int)
merged_test_df['Total_fifties'] = merged_test_df['Total_fifties'].astype(int)
merged_test_df['Runs_Conceded'] = merged_test_df['Runs_Conceded'].astype(int)
merged_test_df['Total_Wickets_taken'] = merged_test_df['Total_Wickets_taken'].astype(int)

In [514]:
merged_test_df = merged_test_df[merged_test_df['Team Name'].isin(team_list)]
merged_test_df.shape

(600, 16)

In [511]:
merged_test_df.head(2)

,Player Name,Team Name,Playing Role,Batting Style,Batting Position,Runs_Scored,Batting Average,Strike_rate,Total_hundreds,Total_fifties,Bowling Style,Runs_Conceded,Total_Wickets_taken,Economy,Bowling Strike Rate,Bowling Average
0,AB de Villiers,South Africa,Wicketkeeper Batter,Right hand Bat,5,5059,57.49,55.65,13,27,Right arm Medium,5,0,5.0,0.0,0.0
1,Aaron Finch,Australia,Top order Batter,Right hand Bat,2,278,27.80,44.98,0,2,Slow Left arm Orthodox,8,0,4.0,0.0,0.0


In [509]:
new_test_player_order = ['Player Name','Team Name','Playing Role','Batting Style','Batting Position','Runs_Scored','Batting Average','Strike_rate','Total_hundreds','Total_fifties','Bowling Style','Runs_Conceded','Total_Wickets_taken','Economy','Bowling Strike Rate','Bowling Average']

In [510]:
merged_test_df = merged_test_df.reindex(columns=new_test_player_order)

In [513]:
merged_test_df = merged_test_df.drop_duplicates(subset=["Player Name","Team Name"])

In [515]:
merged_test_df = merged_test_df[merged_test_df['Player Name'].notna()]

In [521]:
merged_test_df[merged_test_df['Player Name']=='Mitchell Starc']

,Player Name,Team Name,Playing Role,Batting Style,Batting Position,Runs_Scored,Batting Average,Strike_rate,Total_hundreds,Total_fifties,Bowling Style,Runs_Conceded,Total_Wickets_taken,Economy,Bowling Strike Rate,Bowling Average
328,Mitchell Starc,Australia,Bowler,Left hand Bat,9,1493,22.28,68.24,0,10,Left arm Fast,6501,240,3.4,48.1,27.09


In [516]:
merged_test_df.to_csv("merge_test_stats_data.csv",index=False)